# Decoder-only based GPT (language model)

Here we take a transformer block, the decoder in particular, and use it for the task of language modeling. In general, this is how GPTs are trained. We will do this on a much smaller scale.

We take everything we've already built and leverage it in the way Karpathy implements a character level LM here:

In [129]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
!cp drive/MyDrive/Transformers/models/transformer_blocks.py .
!cp drive/MyDrive/Transformers/models/modules.py .
!cp -r drive/MyDrive/Transformers/data/ .

In [131]:
!pip install tokenmonster

In [132]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import random_split
import sys
sys.path.append("~/")
from transformer_blocks import Transformer
from torch.utils.data import Dataset, DataLoader
import tokenmonster

In [133]:
harry_potter_text = " "
for i in range(4):
    book_num = i+1
    with open(f'/content/data/hp{book_num}.txt', 'r', encoding='utf-8') as f:
        harry_potter_text += f.read()
print(len(harry_potter_text))

2652650


## Tokenization
Instead of character level, we're going to model this LM using a tokenizer. in particular, we're going to try to use OpenAI's tiktoken with the gpt2 50k tokenizer. This might end up being too large of a vocab size given compute constraints, but

In [134]:
vocab = tokenmonster.load("fiction-2048-consistent-v1")
tokens = vocab.tokenize("This is a test.")

In [135]:
tokens

array([ 149, 1674,  110,  374,  233,   17], dtype=uint16)

In [136]:
token_example = vocab.tokenize("hello world test monster tokenizer")

In [137]:
token_example

array([  37,  586,  196, 1261,  374,  233,  627,  773,  377,   37,  601,
         53,  252,   62], dtype=uint16)

In [138]:
[vocab.decode([token]) for token in token_example]

['',
 ' hel',
 'lo',
 ' world',
 ' te',
 'st',
 ' mon',
 'ster',
 ' to',
 '',
 ' ken',
 'i',
 'ze',
 'r']

In [139]:
tokens = np.array(vocab.tokenize(harry_potter_text), dtype=np.float16)

In [140]:
dataset = torch.tensor(tokens, dtype=torch.long)
print(dataset.shape, dataset.dtype)

torch.Size([900724]) torch.int64


In [141]:
train_size = int(len(dataset) * 0.8)
test_size = int(len(dataset) * 0.1)
val_size = len(dataset) - train_size - test_size

train_data, test_data, val_data = dataset[:train_size], dataset[train_size:train_size+test_size], dataset[train_size+test_size:]

train_block = torch.tensor([train_data[i] for i in range(100)])
train_list = train_block.tolist()
print(vocab.decode(train_list))

val_block = torch.tensor([val_data[i] for i in range(100)])
val_list = val_block.tolist()
print(vocab.decode(val_list))

test_block = torch.tensor([test_data[i] for i in range(100)])
test_list = test_block.tolist()
print(vocab.decode(test_list))

 Harry Potter and the Sorcerer's Stone


CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such no
 Viktor Krum, the famous International Quidditch player.  It was as though the eighteen-year-old Krum thought he. Harry, was an equal - a real rival -
"You haff never . . . you haff not..."
"No," said Harry very firmly.

moved from his own foot and tricked Mr. Malfoy into giving Dobby, thereby setting Dobby free.  The other was covered in pink and orange stripes.
"Dobby, what're you doing here?" Harry said in amazement. "Dobby has come to work at Hogwarts, sir!" Dob


In [142]:
print(f"train set size: {train_size}, test: {test_size}, val: {val_size}")

train set size: 720579, test: 90072, val: 90073


In [143]:
torch.manual_seed(10000)
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(data):
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # for any sequence x, the target y will be the next 8 tokens
    return x, y

xb, yb = get_batch(train_data)

print(xb.shape, yb.shape)

context = xb[0, :2]
target = yb[0,1]
print(f"when input is {context.tolist()} the target: {target}")

torch.Size([32, 8]) torch.Size([32, 8])
when input is [3, 398] the target: 1034


In [144]:
class HPDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        # Return the total number of possible sequences
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # Fetch a single sequence x and its corresponding target y
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

block_size = 8
train_dataset, val_dataset, test_dataset = HPDataset(train_data, block_size), HPDataset(val_data, block_size), HPDataset(test_data, block_size)

batch_size = 32
train_loader, val_loader, test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True), DataLoader(val_dataset, batch_size=batch_size, shuffle=True), DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [145]:
print(len(train_loader))
print(len(test_loader))
print(len(val_loader))

22518
2815
2815


In [146]:
class zeptoGPT(nn.Module):
    """
    zepto because it's a really small GPT
    """
    def __init__(self, d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size, dropout=0.1) -> None:
        super().__init__()
        self.decoder_transformer = Transformer(d_k, d_model, d_v, d_ff, num_heads, num_layers, vocab_size=vocab_size, mask=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        out = self.decoder_transformer(x)
        return self.fc(out)

In [147]:
def compute_loss(y_target, y_pred, loss_function):
    B, T, C = y_pred.shape
    y_pred = y_pred.view(B*T, C)
    _, max_indices = torch.max(y_pred, dim=1)
    y_target_list = y_target.tolist()
    max_indices = max_indices.tolist()
    y_target = y_target.view(B*T)
    return loss_function(y_pred, y_target)

In [211]:
def generate(model, prompt: str, device,n = 50):
  prompt_array = vocab.tokenize(prompt)
  prompt_array = np.array(prompt_array[:8], dtype=np.int16)
  print(prompt_array.tolist())
  decoded = vocab.decode(prompt_array)
  print(f"prompt: {decoded}")
  for i in range(n):
    prompt_tensor = torch.tensor(prompt_array, dtype=torch.long).to(device)
    next_token = predict_next_token(model, prompt_tensor.unsqueeze(0))
    test_list = next_token.tolist()
    print(vocab.decode(test_list[0]))

In [196]:
def predict_next_token(model, block):
  with torch.no_grad():
    y_pred = model(block)
    token_probs = nn.functional.softmax(y_pred, dim=-1)
    _, max_idx = torch.max(token_probs, dim=-1)

  return max_idx[-1]

In [181]:
def train(model, train_loader, val_loader, loss_function, optim, epochs, device):
    losses = [] #group losses for loss visualization
    running_loss = 0.0
    val_losses = []
    for epoch in range(epochs):
        model.train()
        print("Epoch %d / %d" % (epoch+1, epochs))
        print("-"*10)

        for i, batch_data in enumerate(train_loader):
            x, y = batch_data
            x, y = x.to(device), y.to(device)
            y_pred = model(x)

            loss = compute_loss(y, y_pred, loss_function)
            optim.zero_grad()
            loss.backward()
            optim.step()
            running_loss += loss.item()
            losses.append(loss)

            if (i+1) % 1000 == 0:
                print("Step: {}, average training loss over last 1000 steps: {:.4f}".format(i+1, running_loss/1000))
                running_loss = 0.0

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            correct_pred = 0.0
            num_samples = 0
            for i, batch_data in enumerate(val_loader):
                (y, x) = batch_data
                y, x = y.to(device), x.to(device)
                y_pred = model(x)
                loss = compute_loss(y, y_pred, loss_function)
                _, predicted_labels = torch.max(y_pred, 1)
                num_samples+=predicted_labels.shape[0]
                val_loss += loss.item()

            val_losses.append(val_loss)
        print("Epoch: {}, validation loss: {:.4f}".format(epoch+1, val_loss/len(val_loader)))

    return losses, val_losses

In [182]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 1
DROPOUT = 0.2
D_MODEL = 1024
NUM_HEADS = 8
D_K = int(D_MODEL / NUM_HEADS)
D_V = D_K
D_FF = D_MODEL * 4
NUM_LAYERS = 4
VOCAB_SIZE = vocab.vocab_size
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [183]:
model = zeptoGPT(D_K, D_MODEL, D_V, D_FF, num_heads=NUM_HEADS, num_layers=NUM_LAYERS, vocab_size=VOCAB_SIZE)
model = model.to(DEVICE)

In [184]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [185]:
DEVICE

device(type='cuda', index=0)

In [186]:
train_loss, val_loss = train(model, train_loader, val_loader, torch.nn.functional.cross_entropy, optimizer, NUM_EPOCHS, DEVICE)

Epoch 1 / 1
----------
Step: 1000, average training loss over last 1000 steps: 5.1052
Step: 2000, average training loss over last 1000 steps: 4.6587
Step: 3000, average training loss over last 1000 steps: 4.5170
Step: 4000, average training loss over last 1000 steps: 4.4380
Step: 5000, average training loss over last 1000 steps: 4.3691
Step: 6000, average training loss over last 1000 steps: 4.3248
Step: 7000, average training loss over last 1000 steps: 4.2872
Step: 8000, average training loss over last 1000 steps: 4.2664
Step: 9000, average training loss over last 1000 steps: 4.2211
Step: 10000, average training loss over last 1000 steps: 4.2211
Step: 11000, average training loss over last 1000 steps: 4.1867
Step: 12000, average training loss over last 1000 steps: 4.1623
Step: 13000, average training loss over last 1000 steps: 4.1472
Step: 14000, average training loss over last 1000 steps: 4.1352
Step: 15000, average training loss over last 1000 steps: 4.1238
Step: 16000, average train

In [187]:
text_sample = train_data

In [188]:
print(text_sample[0])
test_block = torch.tensor([text_sample[i] for i in range(8)])

tensor(36)


In [189]:
test_block

tensor([  36,  582,  226,   36,  354,  240,  172, 1528])

In [190]:
test_list = test_block.tolist()

In [191]:
vocab.decode(test_list)

' Harry Potter and the'

In [192]:
test_block = torch.tensor([text_sample[i] for i in range(100)])
test_list = test_block.tolist()
vocab.decode(test_list)

" Harry Potter and the Sorcerer's Stone\n\n\nCHAPTER ONE\n\nTHE BOY WHO LIVED\n\nMr. and Mrs. Dursley, of number four, Privet Drive, were proud to say\nthat they were perfectly normal, thank you very much. They were the last\npeople you'd expect to be involved in anything strange or mysterious,\nbecause they just didn't hold with such no"

In [212]:
generate(model, "This is my text random extra stuff", device= DEVICE)

[149, 1674, 338, 989, 653, 167, 57, 299]
prompt: This is my text random ex
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 go
 exactly
 exactly
 go
 exactly
 exactly
 exactly
 go
 exactly
 exactly
 exactly
 exactly
 exactly
 go
 exactly
 exactly
 exactly
 exactly
 exactly
 go
 go
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 go
 go
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
 exactly
